In [ ]:
# 기본 라이브러리 호출
import warnings
warnings.filterwarnings('ignore')
warnings.warn("once")
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew
# 선형모델을 추정하는 라이브러리
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.stats.api as sms
from patsy import dmatrices
color = sns.color_palette()

pd.set_option('display.float_format','{:,.4f}'.format) # 소수점 2번째 자리까지 표현
pd.set_option('display.max_columns', None) # 모든 컬럼 표시
pd.set_option('display.max_colwidth', -1) # 컬럼내용 전체 표시

#Graph에 한글을 표시하기 위한 코드
import matplotlib
from matplotlib import font_manager, rc
import platform
# font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
font_name = font_manager.FontProperties(fname="/home/spa/.local/share/Trash/files/one/malgun.ttf").get_name()
rc('font', family=font_name)\
    
matplotlib.rcParams['axes.unicode_minus'] = False

# 1. 통계분석 (사용 데이터 : Admission)

### Q1) 종속변수인 chance_of_admit(입학 허가 확률)와 독립변수(GRE, TOEFL, Univ_Rating, SOP, LOR, CGPA)에 대해 피어슨 상관계수를 이용한 상관관계 분석을 수행하고 그래프를 이용하여 분석결과를 설명하시오.

In [ ]:
df = pd.read_csv('./data/Admission.csv')
print(df.shape)
df.head()

In [ ]:
df = df.rename(columns = {'LOR ' : 'LOR'})

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
columns_list = ['GRE', 'TOEFL', 'Univ_Rating', 'SOP', 'LOR', 'CGPA']

In [ ]:
corr_df = pd.DataFrame(columns = ['변수명', '상관계수', 'p-value'])
corr_df['변수명'] = columns_list

In [ ]:
corr_list = []
p_value_list = []
for i in columns_list:
    corr_list.append(stats.pearsonr(df['Chance_of_Admit'], df[i])[0])
    p_value_list.append(stats.pearsonr(df['Chance_of_Admit'], df[i])[1])
corr_df['상관계수'] = corr_list
corr_df['p-value'] = p_value_list
corr_df

* GRE 경우 상관계수는 0.80으로 입학허가확률과 강한 상관관계가 존재. p-value는 0.00로 유의수준 0.05하에서 두 변수간의 상관관계는 통계적으로 유의하다

In [ ]:
corr = df[['Chance_of_Admit', 'GRE', 'TOEFL', 'Univ_Rating', 'SOP', 'LOR', 'CGPA']].corr(method = 'pearson')

In [ ]:
plt.figure(figsize = (6,6))
cmap = sns.light_palette("darkgray", as_cmap=True)
sns.heatmap(df.corr(method = 'pearson'), annot=True, cmap=cmap)
plt.show()

In [ ]:
df.columns

In [ ]:
df.loc[:, 'GRE':'Research']

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(df.drop(columns = ['Research']),
               c = df['Chance_of_Admit'], # 점 색깔
               marker = 'o', # 점 모양, default ‘.’
               s = 10, # 점 크기
               alpha = 0.8, # 투명도
               figsize = (12, 8)
              )
plt.show()

### Q2) GRE, TOEFL, Univ_Rating, SOP, LOR, CGPA, Research가 Chance_of_Admit에 
###     영향을 미치는지 알아보는 회귀분석을 단계적 선택법을 사용하여 수행하고 결과를 해석하시오.

In [ ]:
df.columns

In [ ]:
model = sm.OLS.from_formula('Chance_of_Admit ~ GRE + TOEFL + Univ_Rating + SOP + LOR + CGPA + Research', data = df).fit()
model.summary()

In [ ]:
from statsmodels.stats.stattools import durbin_watson

#perform Durbin-Watson test
durbin_watson(model.resid)

* Durbin-Watson 검정 결과값이 0.76로 0에 가깝기 때문에 독립성 가정을 만족한다고 보기 어렵다

In [ ]:
shapiro_test = stats.shapiro(model.resid)
shapiro_test

* Shapiro-Wilk 검정 결과 p-value가 0.09로 유의수준 0.05하에서 귀무가설을 기각한다. 따라서 adms 데이터는 정규분포를 따른다고 보기 어렵다

In [ ]:
fitted = model.predict(df)
residual = df['Chance_of_Admit'] - fitted

In [ ]:
sns.regplot(fitted, residual, lowess=True, line_kws={'color': 'red'})
plt.plot([fitted.min(), fitted.max()], [0, 0], '--', color='grey')

* 그래프의 기울기가 직선 성향, 평균인 0을 중ㅇ심으로 고르게 분포(등분산성 만족)

In [ ]:
from statsmodels.graphics.gofplots import qqplot
qqplot(model.resid, line='s')
plt.show()

* 대각선에 벗어난 점이 많아 adms가 정규성을 만족한다고 보기 힘듬

In [ ]:
sr = stats.zscore(residual)

In [ ]:
sns.regplot(fitted, np.sqrt(np.abs(sr)), lowess=True, line_kws={'color': 'red'})
plt.xlabel('Fitted_value')
plt.ylabel('sqrt_resid')

* 빨간선의 기울기가 0에 가까워야 하지만 Fitted Value가 증가할수록 기울기가 줄어듬
* 기울기가 0에서 떨어진 점이 있다면 표준화 잔차가 큼, y값 적합 잘 못함

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence
cd, _ = OLSInfluence(model).cooks_distance
cd.sort_values(ascending=False).head()

# 2. 정형 데이터마이닝 (사용 데이터 : Titanic)

### Q1) cabib, embarked변수의 값 중 ""로 처리된 값을 NA로 바꾸고 아래의 데이터 테이블을 보고 
###     문자형, 범주형 변수들을 각각 character, factor형으로 변환하시오.
###     또, 수치형 변수가 NA인 값을 중앙값으로 대체하고, 범주형 변수가 NA인 값을 최빈값으로 대체하고
###     age변수를 아래의 표와 같이 구간화하여 age_1이라는 변수를 생성하고 추가하시오. 

In [ ]:
df = pd.read_csv('./data/titanic.csv')
print(df.shape)
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df['embarked'].unique()

In [ ]:
df.info()

In [ ]:
df['pclass'] = df['pclass'].astype('category')
df['survived'] = df['survived'].astype('category')

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
age_med = np.nanmedian(df['age'])

In [ ]:
age_med

In [ ]:
df['age'] = df['age'].fillna(value = age_med)

In [ ]:
fare_med = np.nanmedian(df['fare'])

In [ ]:
df['fare'] = df['fare'].fillna(value = fare_med)

In [ ]:
from collections import Counter as cc
cnt = cc(df['cabin'])
cnt.most_common()[1][0]

In [ ]:
df['cabin'] = df['cabin'].fillna(value = cnt.most_common()[1][0])

In [ ]:
from collections import Counter as cc
cnt = cc(df['embarked'])
cnt.most_common()[:10]

In [ ]:
df['embarked'] = df['embarked'].fillna(value = 'S')

In [ ]:
df.loc[((df['age'] >= 0) & (df['age'] < 10)), 'age_1'] = 0
df.loc[((df['age'] >= 10) & (df['age'] < 20)), 'age_1'] = 1
df.loc[((df['age'] >= 20) & (df['age'] < 30)), 'age_1'] = 2
df.loc[((df['age'] >= 30) & (df['age'] < 40)), 'age_1'] = 3
df.loc[((df['age'] >= 40) & (df['age'] < 50)), 'age_1'] = 4
df.loc[((df['age'] >= 50) & (df['age'] < 60)), 'age_1'] = 5
df.loc[((df['age'] >= 60) & (df['age'] < 70)), 'age_1'] = 6
df.loc[((df['age'] >= 70) & (df['age'] < 80)), 'age_1'] = 7
df.loc[((df['age'] >= 80) & (df['age'] < 90)), 'age_1'] = 8


In [ ]:
df['age_1'].value_counts()

In [ ]:
df.head()

### Q2) 전처리가 완료된 titanic 데이터를 train(70%), test(30%) 데이터로 분할하시오.
###    (set.seed(12345)를 실행한 후 데이터를 분할하시오.) 
###    또, train 데이터로 종속변수인 survived(생존 여부)를 독립변수 pclass, sex, sibsp, parch, 
###    fare, embarked로 지정하여 예측하는 분류모델을 3개 이상 생성하고 test 데이터에 대한 
###    예측값을 csv파일로 각각 제출하시오.

In [ ]:
#!pip install xgboost

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
lr = LogisticRegression()
rf = RandomForestClassifier()
dt = DecisionTreeClassifier()
xgb = XGBClassifier()

In [ ]:
X = df[['pclass', 'sex', 'sibsp', 'parch', 'fare', 'embarked']]
y = df['survived']

In [ ]:
X = pd.get_dummies(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 6789)

In [ ]:
df.isna().sum()

In [ ]:
lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)
pd.Series(lr_y_pred).value_counts()

In [ ]:
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
pd.Series(rf_y_pred).value_counts()

In [ ]:
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)
pd.Series(xgb_y_pred).value_counts()

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, lr_y_pred)
print('오차 행렬:\n', confusion)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, lr_y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, rf_y_pred)
print('오차 행렬:\n', confusion)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf_y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, xgb_y_pred)
print('오차 행렬:\n', confusion)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, xgb_y_pred))

### Q3) 생성된 3개의 분류모델에 대해 성과분석을 실시하여 정확도를 비교하여 설명하시오. 
###     또, ROC curve를 그리고 AUC값을 산출하시오.

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, lr.decision_function(X_test))

plt.plot(fpr, tpr, label = 'ROC 곡선')
plt.xlabel('FPR')
plt.ylabel('TPR (재현율)')
# 0 근처의 임계값을 찾습니다.
close_zero = np.argmin(np.abs(thresholds))
plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize = 10,
        label = '임계값 0', fillstyle = 'none', c = 'k', mew = 2)
plt.legend(loc = 4)

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, lr.decision_function(X_test))

plt.plot(fpr, tpr, label = 'ROC 곡선')
plt.xlabel('FPR')
plt.ylabel('TPR (재현율)')
# 0 근처의 임계값을 찾습니다.
# close_zero = np.argmin(np.abs(thresholds))
# plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize = 10,
#         label = '임계값 0', fillstyle = 'none', c = 'k', mew = 2)
plt.legend(loc = 4)

In [ ]:
from sklearn.metrics import roc_auc_score
lr_auc = roc_auc_score(y_test, lr.predict_proba(X_test)[:, 1])
# svc_auc = roc_auc_score(y_test, svc.decision_function(X_test))
print('lr의 AUC: {:.3f}'.format(lr_auc))
# print('SVC의 AUC: {:.3f}'.format(svc_auc))

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, rf.predict_proba(X_test)[:, 1])

plt.plot(fpr, tpr, label = 'ROC 곡선')
plt.xlabel('FPR')
plt.ylabel('TPR (재현율)')
# 0 근처의 임계값을 찾습니다.
close_zero = np.argmin(np.abs(thresholds))
plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize = 10,
        label = '임계값 0', fillstyle = 'none', c = 'k', mew = 2)
plt.legend(loc = 4)

In [ ]:
from sklearn.metrics import roc_auc_score
rf_auc = roc_auc_score(y_test, rf.predict_proba(X_test)[:, 1])
# svc_auc = roc_auc_score(y_test, svc.decision_function(X_test))
print('랜덤 포레스트의 AUC: {:.3f}'.format(rf_auc))
# print('SVC의 AUC: {:.3f}'.format(svc_auc))

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, xgb.predict_proba(X_test)[:, 1])

plt.plot(fpr, tpr, label = 'ROC 곡선')
plt.xlabel('FPR')
plt.ylabel('TPR (재현율)')
# 0 근처의 임계값을 찾습니다.
close_zero = np.argmin(np.abs(thresholds))
plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize = 10,
        label = '임계값 0', fillstyle = 'none', c = 'k', mew = 2)
plt.legend(loc = 4)

In [ ]:
from sklearn.metrics import roc_auc_score
xgb_auc = roc_auc_score(y_test, xgb.predict_proba(X_test)[:, 1])
# svc_auc = roc_auc_score(y_test, svc.decision_function(X_test))
print('xgboost의 AUC: {:.3f}'.format(xgb_auc))
# print('SVC의 AUC: {:.3f}'.format(svc_auc))

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, xgb_y_pred)

# 3. 비정형 데이터마이닝 (사용 데이터 : 문재인대통령 취임사)

### 1) ‘연설문.txt’ 데이터를 읽어온 뒤 숫자, 특수 문자 등을 제거하는 전처리 작업을 시행하시오.

In [ ]:
def read_data(filename):
    with open(filename, 'r', encoding = 'cp949') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

In [ ]:
text = read_data('./data/연설문.txt')

In [ ]:
text = ' '.join(x for x in np.array(text).reshape(-1,))

In [ ]:
text[:100]

In [ ]:
import konlpy
import re

In [ ]:
def cleansing(text):
    repl = ''
    pattern  = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern = pattern, repl = repl, string = text)
    pattern  = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern = pattern, repl = repl, string = text)
    pattern  = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern = pattern, repl = repl, string = text)
    pattern  = '<[^>]*>'
    text = re.sub(pattern = pattern, repl = repl, string = text)
    pattern = '[^\w\s]'
    text = re.sub(pattern = pattern, repl = repl, string = text)
    return text    

In [ ]:
text = cleansing(text)

In [ ]:
from konlpy.tag import Komoran
komoran = Komoran()
nouns = komoran.nouns(text)

In [ ]:
from collections import Counter
cnt = Counter(nouns)

In [ ]:
temp_df = pd.DataFrame(columns = ['명사', '빈도'])
freq = cnt.most_common(10)
temp_list = []
temp_list2= []
for i in freq:
    temp_list.append(i[0])
    temp_list2.append(i[1])
temp_df['명사'] = temp_list
temp_df['빈도'] = temp_list2
temp_df

In [ ]:
plt.figure(figsize = (8,6))
sns.barplot(x = '명사', y = '빈도', data = temp_df)
plt.tight_layout()
plt.show()

In [ ]:
words = {}
for i in cnt.most_common(20):
    words[i[0]] = i[1]   

In [ ]:
from wordcloud import WordCloud 
import nltk
from nltk.corpus import stopwords

wordcloud = WordCloud(font_path = "/usr/share/fonts/NanumGothicCoding.ttf", background_color='white',colormap = "Accent_r",
                      width=1500, height=1000).generate_from_frequencies(words)

plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
# wordcloud = WordCloud(font_path = 'C:/Windows/Fonts/malgun.ttf', background_color='white',colormap = "Accent_r",
#                       width=1500, height=1000).generate_from_frequencies(words)
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.show()